In [ ]:
import hashlib

In [ ]:
# Utility functions

lst = {'a': 97,
 'b': 98,
 'c': 99,
 'd': 100,
 'e': 101,
 'f': 102,
 'g': 103,
 'h': 104,
 'i': 105,
 'j': 106,
 'k': 107,
 'l': 108,
 'm': 109,
 'n': 110,
 'o': 111,
 'p': 112,
 'q': 113,
 'r': 114,
 's': 115,
 't': 116,
 'u': 117,
 'v': 118,
 'w': 119,
 'x': 120,
 'y': 121,
 'z': 122}

def string_to_int(s):
    n  = 0
    i = 0
    for c in s:
        tmp = lst[c]
        n = n + tmp*(256**i)
        i += 1
    return n

def int_to_string(n):
    s = ""
    while n > 0:
        a = n%256
        s += chr(a) 
        n = n//256
    return s

def fast_power(g, A):
    a = g
    b = 1
    if g == 0 and A == 0:
        return "Undefined"
    else:
        while A > 0:
            if A%2 == 1:
                b = b*a
            a = a^2
            A = A//2
            print
        return b

In [ ]:
# Parameters are defined here

p = 7691 # prime
k = 2 # Embedding degree
F_p = GF(p) # Field
F_p2 = GF(p**2) # Extension field
E = EllipticCurve(F_p, [0,1]) # E : y^2 = x^3 + 1
E_2 = EllipticCurve(F_p2, [0, 1])
r = 641
l = (p + 1)//r
P = E(0)
s = 0
P_pub = P

In [ ]:
def h1(text):
    hash = sha1()
    str_val = str(text)
    byte_val = str_val.encode()
    hash.update(byte_val)
    hexadecimal = hash.hexdigest()
    return int(hexadecimal, 16)

In [ ]:
def get_params():
    while P == E(0):
        Q = E.random_element()
        while Q == E(0): 
            Q = E.random_element()
        P = l*Q 
    s = ZZ.random_element(2,q-1) 
    P_pub = s*P
    public = [p, r, P, l, E, P, P_pub] #public parameters
    private = s #secret
    return public, private    

In [ ]:
def g(P,Q,x,y,E):
    A,B = E.a_invariants()[3], E.a_invariants()[4]
    if P == E(0) or Q == E(0):
        return "no divisor"
    xP,yP = P[0],P[1]
    xQ,yQ = Q[0],Q[1]
    if yP == -yQ and xP == xQ:
        slope = +oo #symbol for Infinity
    elif P == Q:
        slope = (3*(xP**2) + A)/(2*yP)
    else:
        slope = (yQ - yP)/(xQ - xP)
    #return the function on E
    if slope == +oo:
        return x - xP
    else:
        return (y - yP - slope*(x - xP))/(x + xP + xQ - slope**2)
    
    
def MillerAlgorithm(P,m,x,y,E):
    A,B = E.a_invariants()[3], E.a_invariants()[4]
    xP,yP = P[0],P[1]
    binary = m.digits(2) 
    n = len(binary) 
    T = P
    f = 1
    for i in range(n-2,-1,-1): 
        f = (f**2)*g(T,T,x,y,E)
        T *= 2 
        if binary[i] == 1:
            f = f*g(T,P,x,y,E)
            T += P
    return f

def WeilPairing(P,Q,m,E):
    A,B = E.a_invariants()[3], E.a_invariants()[4]
    S = E.random_element()
    while m*S == E(0):
        S = E.random_element() 
    xS,yS = S[0],S[1]
    QplusS = Q + S
    f_P_QplusS = MillerAlgorithm(P,m,QplusS[0],QplusS[1],E)
    f_P_S = MillerAlgorithm(P,m,xS,yS,E)
    num = f_P_QplusS/f_P_S

    PminusS = P - S
    f_Q_PminusS = MillerAlgorithm(Q,m,PminusS[0],PminusS[1],E) 
    f_Q_minusS = MillerAlgorithm(Q,m,xS,-yS,E) 
    denom = f_Q_PminusS/f_Q_minusS

    e_m = num/denom
    return e_m

def ModifiedWeilPairing(P,Q,m,E):
    Fp = GF(p)
    R.<x> = Fp[]
    Fp2.<z> = Fp.extension(x^2+x+1)      
    E_zeta = EllipticCurve(Fp2, [0,1])   
    phiQ = E_zeta([z*Q[0],Q[1]])
    A,B = E_zeta.a_invariants()[3], E_zeta.a_invariants()[4]
    P_zeta = E_zeta([P[0],P[1]])
    S = E_zeta.random_element()
    while m*S == E(0):
        S = E_zeta.random_element()
    xS,yS = S[0],S[1]
    QplusS = phiQ + S
    f_P_QplusS = MillerAlgorithm(P,m,QplusS[0],QplusS[1],E_zeta)
    f_P_S = MillerAlgorithm(P,m,xS,yS,E_zeta)
    num = f_P_QplusS/f_P_S

    PminusS = P_zeta - S
    f_Q_PminusS = MillerAlgorithm(phiQ,m,PminusS[0],PminusS[1],E_zeta) 
    f_Q_minusS = MillerAlgorithm(phiQ,m,xS,-yS,E_zeta) #f_Q(-S)
    denom = f_Q_PminusS/f_Q_minusS

    e_m = num/denom
    return e_m

In [ ]:
def id_to_point(hh):
    xx = pow((hh^2) - 1, ((2*p)-1)//3, p) 
    Q = E([xx, hh])
    pub_ID = l*Q 
    priv_ID = s*pub_ID
    return pub_ID, priv_ID

In [ ]:
def encrypt(msg, pub_ID):
    # params = [p, q, l, E, P, P_pub]
    rand = ZZ.random_element(2, r-1)
    U = rand*P
    g_ID = ModifiedWeilPairing(pub_ID, P_pub, r, E)
    g_ID_to_rand = fast_power(g_ID, rand)
    V = string_to_int(msg)^^h1(g_ID_to_rand)
    C = [U, V]
    return C

In [ ]:
def decrypt(cipher_text, priv_ID):
    U, V = cipher_text
    weil = ModifiedWeilPairing(priv_ID, U, r, E)
    M = V^^h1(weil)
    return M

In [ ]:
my_id = "rachit"
hh = h1(my_id)
pub_ID, priv_ID = id_to_point(hh)

cipher_text = encrypt("hello", pub_ID)
decrypted_text = int_to_string(decrypt(cipher_text, priv_ID))

print(decrypted_text)